In [1]:
! pip install "magentic[anthropic]"
! pip install dspy-ai PyPDF2


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
!pwd

/Users/jaidhyani/Desktop/atefar


In [15]:
! pip uninstall -y atefar
! pip install -e .

Found existing installation: atefar 0.1.0
Uninstalling atefar-0.1.0:
  Successfully uninstalled atefar-0.1.0
Obtaining file:///Users/jaidhyani/Desktop/atefar
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for atefar (pyproject.toml) ... done
  Created wheel for atefar: filename=atefar-0.1.0-py2.py3-none-any.whl size=895 sha256=8608e0a9242dc90429748827ec12d87c93c52c602affb7a176e1a04597151e0a
  Stored in directory: /private/var/folders/5k/7nfpl0cs5999pzhndyybcn800000gn/T/pip-ephem-wheel-cache-v66pt9bs/wheels/ef/a2/67/4e2ac8e515272dc3b0b25ecb48e15da2a9a7bc08f6adb925d8
Successfully built atefar


In [4]:
# update requirements.txt
! pip freeze > requirements.txt

In [14]:
!pip list

Package            Version     Editable project location
------------------ ----------- -------------------------------
aiohappyeyeballs   2.3.5
aiohttp            3.10.2
aiosignal          1.3.1
alembic            1.13.2
annotated-types    0.7.0
anthropic          0.32.0
anyio              4.4.0
appnope            0.1.4
asttokens          2.4.1
atefar             0.1.0       /Users/jaidhyani/Desktop/atefar
attrs              24.2.0
backoff            2.2.1
certifi            2024.7.4
charset-normalizer 3.3.2
colorlog           6.8.2
comm               0.2.2
datasets           2.20.0
debugpy            1.8.5
decorator          5.1.1
dill               0.3.8
distro             1.9.0
dspy-ai            2.4.13
executing          2.0.1
filelock           3.15.4
filetype           1.2.0
frozenlist         1.4.1
fsspec             2024.5.0
h11                0.14.0
httpcore           1.0.5
httpx              0.27.0
huggingface-hub    0.24.5
idna               3.7
ipykernel          6.29.5
ip

In [9]:
import sys
print(sys.path) 

['/Users/jaidhyani/miniforge3/envs/mp4/lib/python311.zip', '/Users/jaidhyani/miniforge3/envs/mp4/lib/python3.11', '/Users/jaidhyani/miniforge3/envs/mp4/lib/python3.11/lib-dynload', '', '/Users/jaidhyani/Desktop/atefar/.venv/lib/python3.11/site-packages']


In [1]:
import dspy

import atefar
from atefar import paper_analysis, pdf_utils

/Users/jaidhyani/Desktop/atefar/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [78]:
from dataclasses import dataclass
from dspy.primitives.program import Module
from typing import Any, Optional


@dataclass
class SigData:
    name: str
    desc: str

@dataclass
class SigStep:
    outputs: list[SigData]
    module: Module = dspy.ChainOfThought
    kwargs: Optional[dict[str, Any]] = None

@dataclass
class SigChain:
    inputs: list[SigData]
    steps: list[SigStep]

paper_chain = SigChain(
    inputs=[SigData("paper_content", "The full text content of the research paper")],
    steps=[
        SigStep([
                SigData("title", "The title of the paper"),
                SigData("abstract_plus", "The abstract of the paper, plus optional additional high-level summaries to capture interesting aspects of the paper not otherwise covered in the abstract"),
        ],),
        SigStep([
            SigData(
                "core_ideas", 
                """
                Enumerate the key innovative ideas in the paper that led to specific, implementable improvements over baseline methods. For each idea:
                1. Briefly describe the idea
                2. Explain how it improves upon a baseline approach
                3. Mention the specific metric(s) it aims to improve

                Example:
                1. Idea: Skip connections in neural networks
                - Description: Direct connections between non-adjacent layers
                - Improvement: Allows better gradient flow in deep networks
                - Target metrics: Training speed, final accuracy
                """
            ),
        ],),
        SigStep([
            SigData(
                "metrics",
                """
                For each metric used in the paper:
                1. Provide a clear, implementable definition
                2. Specify the exact calculation method
                3. Describe the typical range of values and what they indicate
                4. Note any specific conditions or datasets used for measurement
                5. List which experimental methods target this metric
                6. Provide a simple Python function to calculate the metric, taking only the experimental method as an argument

                Example 1: Perplexity
                1. Definition: A measurement of how well a probability model predicts a sample, used in NLP to evaluate language models.
                2. Calculation: Perplexity = exp(cross_entropy_loss)
                3. Typical range: 
                    - Lower is better
                    - For word-level language models on PTB: 50-100 is good, <60 is state-of-the-art
                4. Conditions: Calculated on held-out test data, using the Penn Treebank (PTB) dataset
                5. Targeted by: 
                    - Experimental Method 1: Sparse Attention
                    - Experimental Method 3: Adaptive Layer Normalization
                6. Python function:
                ```python
                import torch
                import torch.nn.functional as F

                # Preset constants
                test_data = load_ptb_test_data()  # Function to load PTB test data
                vocab_size = 10000  # PTB vocabulary size

                def calculate_perplexity(experimental_method):
                    model = experimental_method()  # Assuming the method returns an instantiated model
                    model.eval()
                    total_loss = 0
                    total_tokens = 0
                    with torch.no_grad():
                        for batch in test_data:
                            inputs, targets = batch
                            outputs = model(inputs)
                            loss = F.cross_entropy(outputs.view(-1, vocab_size), targets.view(-1), reduction='sum')
                            total_loss += loss.item()
                            total_tokens += targets.numel()
                    return torch.exp(total_loss / total_tokens)
                ```

                Example 2: BLEU Score (for machine translation)
                1. Definition: Bilingual Evaluation Understudy, measures the similarity between machine-generated translations and reference translations.
                2. Calculation: Geometric mean of n-gram precisions (typically n=1 to 4), with a brevity penalty.
                3. Typical range: 
                    - 0 to 100, higher is better
                    - 30-50 is considered good for many language pairs
                4. Conditions: Calculated on the WMT14 English-German test set
                5. Targeted by:
                    - Experimental Method 2: Multi-head Attention Pruning
                    - Experimental Method 4: Gradient Accumulation Training
                6. Python function:
                ```python
                from nltk.translate.bleu_score import corpus_bleu

                # Preset constants
                test_data = load_wmt14_en_de_test_data()  # Function to load WMT14 En-De test data
                reference_translations = load_reference_translations()  # Function to load reference translations

                def calculate_bleu(experimental_method):
                    model = experimental_method()  # Assuming the method returns an instantiated model
                    hypotheses = []
                    for source_sentence in test_data:
                        translation = model.translate(source_sentence)
                        hypotheses.append(translation)
                    return corpus_bleu([[ref] for ref in reference_translations], hypotheses) * 100
                ```

                Example 3: Inference Speed (tokens/second)
                1. Definition: The number of tokens processed per second during model inference.
                2. Calculation: (Number of tokens in test set) / (Total inference time)
                3. Typical range:
                    - Higher is better
                    - Varies greatly depending on model size and hardware; baseline might be 1000 tokens/sec
                4. Conditions: Measured on a single NVIDIA V100 GPU, using a fixed test set of 10,000 sentences
                5. Targeted by:
                    - Experimental Method 1: Sparse Attention
                    - Experimental Method 5: Quantization-Aware Training
                6. Python function:
                ```python
                import time

                # Preset constants
                test_sentences = load_test_sentences()  # Function to load 10,000 test sentences
                num_tokens = sum(len(sentence.split()) for sentence in test_sentences)

                def measure_inference_speed(experimental_method):
                    model = experimental_method()  # Assuming the method returns an instantiated model
                    model.eval()
                    start_time = time.time()
                    with torch.no_grad():
                        for sentence in test_sentences:
                            _ = model(sentence)
                    total_time = time.time() - start_time
                    return num_tokens / total_time
                ```
                """
            ),
        ],),
        SigStep([
            SigData(
                "baseline_methods", 
                """
                    Describe the key baseline methods that the paper's improvements build upon. For each method:
                        1. Provide a brief, high-level description
                        2. List the key components or steps of the method
                        3. Mention any specific architectural details, hyperparameters, or algorithmic choices that are crucial for implementation
                        4. Note which experimental methods modify this baseline

                    Example:
                        Baseline Method: Vanilla Transformer
                            1. Description: A sequence-to-sequence model based on self-attention mechanisms
                            2. Key components:
                                - Multi-head self-attention layers
                                - Feed-forward neural networks
                                - Layer normalization
                                - Positional encodings
                            3. Crucial details:
                                - 6 encoder and 6 decoder layers
                                - 8 attention heads
                                - 512-dimensional embeddings
                                - Learning rate warmup over 4000 steps
                            4. Modified by: Experimental methods 1 (sparse attention) and 2 (adaptive layer normalization)
                """
            ),
        ],),
        SigStep([
            SigData(
                "experimental_methods", 
                """
                Enumerate the key improvements or modifications proposed in the paper. For each method:
                    1. Identify the baseline method it modifies
                    2. Describe the specific changes or additions to the baseline
                    3. Explain how these changes are expected to improve performance
                    4. List the metrics this modification aims to improve

                Example 1:
                    1. Baseline: Vanilla Transformer
                    2. Modification: Sparse Attention Mechanism
                        - Replace full attention with sparse attention patterns
                        - Implement fixed or learned attention patterns
                    3. Expected improvements:
                        - Reduced computational complexity from O(n²) to O(n log n)
                        - Better handling of long sequences
                    4. Target metrics: Inference speed, memory usage, performance on long-sequence tasks

                Example 2:
                    1. Baseline: Vanilla Transformer
                    2. Modification: Adaptive Layer Normalization
                        - Replace fixed layer norm with input-dependent dynamic scaling
                        - Implement a small network to predict normalization parameters
                    3. Expected improvements:
                        - Better adaptation to varying input distributions
                        - Improved gradient flow in deep networks
                    4. Target metrics: Training convergence speed, final model accuracy
                """
            ),
        ],),
        SigStep([
            SigData(
                "method_metric_results", 
                """
                For each baseline and experimental method, as well as combinations thereof, list the metric results that were reported in the paper. These should be quantitative results used to compare the methods to each other.
                """
            ),
        ],),
        SigStep([
            SigData(
                "hw_agnostic_metrics", 
                """
                List metrics that correspond to the paper's metrics, but are not hardware-specific. These may be copied from input metrics where appropriate, or they may be generated if not otherwise available. 
                For example, if the paper reports time to train on a specific hardware setup, a hardware-agnostic metric might be number of samples used for training, or total FLOPS used to train to the same level of accuracy.
                Hardware agnostic metrics should be closely correlated with the hardware-specific metrics and indicate the same relative performance of different methods.
                """
            ),
        ],),
        SigStep([
            SigData(
                "tasks", 
                """
                For each experimental method, describe a programming task that implements the improvement. Each task should:
                    1. Identify the baseline implementation to be modified
                    2. Clearly state the specific modification or addition to be made
                    3. Specify the inputs and outputs of the function, including types
                    4. Describe any key constraints or requirements for the implementation
                    5. Mention how the improvement's effect could be measured or observed in the output

                Example:
                    Task: Implement Sparse Attention in Transformer
                        1. Baseline: transformer.py containing a standard Transformer implementation
                        2. Modification: Implement a sparse attention mechanism in the MultiHeadAttention class
                        3. Inputs/Outputs:
                            - Input: torch.Tensor of shape (batch_size, seq_length, d_model)
                            - Output: torch.Tensor of same shape as input
                        4. Key requirements:
                            - Implement a fixed sparse attention pattern (e.g., local + global attention)
                            - Ensure the sparse attention matrix is properly masked and normalized
                            - Maintain compatibility with the rest of the Transformer architecture
                        5. Observable effects:
                            - Reduced memory usage for attention weights
                            - Faster computation time for forward and backward passes
                            - Similar or improved perplexity on language modeling tasks
                """
            ),
        ],),
        SigStep([
            SigData(
                "task_eval_planning",
                """

                For each task, plan an evaluation that tests the engineer's ability to implement the specific improvement. Include:
                    1. The baseline implementation to be provided
                    2. A clear description of the required modification
                    3. Sample inputs to test both the original functionality and the improvement
                    4. Expected changes in the output that demonstrate the improvement
                    5. Metrics to measure the implementation's correctness and performance gain

                Example:
                    Eval Plan: Sparse Attention Implementation
                        1. Baseline: Provide transformer.py with standard MultiHeadAttention class
                        2. Required modification: Implement sparse attention with local+global pattern
                        3. Sample inputs:
                            - Short sequence (length 50) to test preserved functionality
                            - Long sequence (length 1000) to test improvement
                        4. Expected changes:
                            - Similar output quality for short sequences
                            - Improved quality or efficiency for long sequences
                            - Sparse attention weight matrices in intermediate outputs
                        5. Evaluation metrics:
                            - Correctness: Compare attention patterns with expected sparsity
                            - Performance: Measure speed and memory usage improvements
                            - Quality: Compare perplexity on a held-out test set
                """
            ),
        ],),
        SigStep([
            SigData(
                "task_eval_baseline_implementation", 
                """
                For each task eval, if applicable, provide a baseline implementation that will be given to the engineer to modify. This should be in python. If any ML is required, 
                assume that the engineer has access to a python environment with GPUs, and prefer to use torch for ML tasks. Other common libraries may be used as well, but
                the implementation should be as simple as possible. The baseline implementation should be as close to the paper's method as possible, and allow the engineer 
                to focus on the specific modification required by the task. If this task does not require a baseline implementation, simply indicate that instead.
                """
            ),
        ],),
        SigStep([
            SigData(
                "task_eval_instructions", 
                """
                For each enumerated task, describe the instructions that will be given to the engineer to complete the task. Instructions should be clear and concise,
                identify the inputs and outputs, and provide any additional information that the engineer will need to complete the task. If the task involves modifying a baseline implementation,
                the instructions should clearly describe the modification to be made, and may refer to the baseline implementation.
                """
            ),
        ]),
        SigStep([
            SigData(
                "task_eval_correctness_scoring", 
                """
                For each task, write a Python function to score the correctness of the implementation. This function should:
                    1. Take as input the engineer's modified function and the original baseline
                    2. Test both the preserved baseline functionality and the specific improvement
                    3. Check for the presence and correct implementation of the new feature
                    4. Verify that unrelated aspects of the baseline remain unchanged
                    5. Return a score between 0 and 1, with guidelines for partial credit

                Example:
                    ```python
                    def score_sparse_attention_implementation(modified_transformer, baseline_transformer):
                        score = 0.0
                        
                        # Test preserved functionality
                        short_input = torch.randn(32, 50, 512)
                        if torch.allclose(modified_transformer(short_input), baseline_transformer(short_input), atol=1e-5):
                            score += 0.3
                        
                        # Test improvement
                        long_input = torch.randn(32, 1000, 512)
                        modified_output = modified_transformer(long_input)
                        
                        # Check for sparse attention pattern
                        attention_weights = modified_transformer.encoder.layers[0].self_attn.attn_weights
                        if attention_weights.float().to_dense().count_nonzero() / attention_weights.numel() < 0.2:
                            score += 0.4
                        
                        # Check for improved efficiency
                        start_time = time.time()
                        modified_transformer(long_input)
                        modified_time = time.time() - start_time
                        
                        start_time = time.time()
                        baseline_transformer(long_input)
                        baseline_time = time.time() - start_time
                        
                        if modified_time < 0.8 * baseline_time:
                            score += 0.3
                        
                        return score
                    ```
                """
            ),
        ]),
        SigStep([
            SigData(
                "task_eval_metric_scoring", 
                """
                For each task eval, write a python function that will be used to score the metric results of the implementation. The input will be the function written by the engineer,
                and the output will be a score between 0 and 1 indicating how well the implementation matches the expected results. This will typically be completed by comparing the output of the
                implementation to the output of the method in the paper for some known input. For example, if the task was to modify a neural-network-building function to add skip connections,
                the scoring function should check whether the improvement of the modified network over the baseline network matches the expected improvement. 
                """
            ),
        ]),
        SigStep([
            SigData(
                "implementation_difficulty", 
                """
                For each task, estimate the difficulty of implementing the improvement:
                    1. Rate the complexity on a scale of 1-5
                    2. Estimate the time required for an average ML engineer to implement
                    3. List any specific challenges or potential pitfalls
                    4. Suggest any helpful resources or references for implementation

                Example 1: Sparse Attention Implementation
                    1. Complexity: 4/5
                    2. Estimated time: 4-6 hours
                    3. Challenges and pitfalls:
                        - Efficient implementation of sparse matrix operations
                        - Ensuring correct backpropagation through sparse operations
                        - Maintaining numerical stability with sparse normalization
                    4. Helpful resources:
                        - "Generating Long Sequences with Sparse Transformers" (Child et al., 2019)
                        - PyTorch Sparse module documentation
                        - "Longformer: The Long-Document Transformer" (Beltagy et al., 2020)

                Example 2: Adaptive Layer Normalization
                1. Complexity: 3/5
                2. Estimated time: 2-3 hours
                3. Challenges and pitfalls:
                    - Ensuring the adaptive parameters are properly initialized
                    - Avoiding overfitting in the normalization parameter prediction
                    - Maintaining overall model stability during training
                4. Helpful resources:
                    - "Layer Normalization" (Ba et al., 2016)
                    - PyTorch nn.LayerNorm implementation
                    - "Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift" (Ioffe & Szegedy, 2015)
                """
            ),
        ],),
        SigStep([
            SigData(
                "data_processing",
                """
                Describe any specific data processing techniques used in the paper:
                1. Data cleaning or preprocessing steps
                2. Feature engineering methods
                3. Data augmentation techniques
                4. Sampling or batching strategies

                Example 1: NLP Preprocessing for Transformer Model
                1. Data cleaning:
                    - Lowercase all text
                    - Remove special characters and extra whitespace
                    - Tokenization using SentencePiece with vocab size 32,000
                2. Feature engineering:
                    - Convert tokens to integer IDs
                    - Add special tokens: [CLS] at start, [SEP] at end of each sentence
                3. Data augmentation:
                    - Random word masking (15% of tokens)
                    - Random word replacement (10% of tokens)
                4. Batching strategy:
                    - Group sentences of similar length
                    - Pad to max length in batch
                    - Use attention mask to ignore padding

                Example 2: Image Processing for CNN
                1. Data cleaning:
                    - Remove corrupted images
                    - Resize all images to 224x224 pixels
                2. Feature engineering:
                    - Normalize pixel values to range [0, 1]
                    - Subtract mean and divide by std dev (per channel)
                3. Data augmentation:
                    - Random horizontal flips (50% probability)
                    - Random crops (224x224 from 256x256 padded image)
                    - Color jitter (brightness, contrast, saturation)
                4. Batching strategy:
                    - Random sampling
                    - Batch size of 32 for training, 64 for validation
                """
) 
        ]),            
        SigStep([
            SigData(
                "setup_description", 
                """
    Provide a detailed setup guide:
        1. List all required libraries and their minimum versions (and maximum only if necessary)
        2. Describe the necessary compute resources (GPU, RAM, etc.)
        3. Specify any datasets, including versions and how to access them
        4. Include a sample environment setup script
        5. Note any potential compatibility issues or common setup pitfalls

    Example:
    1. Required libraries:
       - PyTorch>=1.9.0
       - torchvision>=0.10.0
       - transformers>=4.9.2
       - numpy>=1.21.2
       - pandas>=1.3.3
       - matplotlib>=3.4.3
    2. Compute resources:
       - GPU: NVIDIA V100 or A100 (at least 16GB VRAM)
       - RAM: 32GB minimum, 64GB recommended
       - Storage: 100GB free space for datasets and model checkpoints
    3. Datasets:
       - ImageNet (ILSVRC2012), available from image-net.org
       - COCO 2017, available from cocodataset.org
    4. Environment setup:
    ```bash
    # Create and activate conda environment
    conda create -n paper_env python=3.8
    conda activate paper_env

    # Install PyTorch with CUDA support
    conda install pytorch torchvision cudatoolkit=11.1 -c pytorch -c nvidia

    # Install other dependencies
    pip install transformers numpy pandas matplotlib

    # Set up data directories
    mkdir -p data/{imagenet,coco}
    # Download datasets to respective directories
    ```
    5. Potential issues:
       - CUDA version mismatch: Ensure CUDA toolkit version matches PyTorch build
       - OOM errors: Reduce batch size or use gradient accumulation for large models
       - Dataset download issues: Use academic credentials for ImageNet access
                
                
                A description of the setup required before implementing the task functions. This should include a list of assets that need to be available to start.
                These may include datasets, trained models, training functions, etc. Assume that a python torch environment with GPUs is available. Any common libraries
                or public assets may be used (e.g. huggingface transformers, torchvision, etc.). If a dataset is required, it should be available in a public location.
                """
            ),
        ],),
    ],
)


def solve_sigchain(chain: SigChain, inputs=dict[str, str]) -> tuple[dict[str, Any], dict[str, Any]]:
    input_sigs = chain.inputs.copy()
    input_vals = inputs.copy()
    solvers = dict()
    results = dict()
    for step in chain.steps:
        step_name = "_".join([o.name for o in step.outputs])
        step_inputs_str = ", ".join([i.name for i in input_sigs])
        step_outputs_str = ", ".join([o.name for o in step.outputs])
        print(f"Running step {step_name}")
        print(f"  Inputs: {step_inputs_str}")
        print(f"  Outputs: {step_outputs_str}")
        class NextSig(dspy.Signature):
            pass
        for i in input_sigs:
            NextSig = NextSig.append(i.name, dspy.InputField(desc=i.desc))
        for o in step.outputs:
            NextSig = NextSig.append(o.name, dspy.OutputField(desc=o.desc))
        step_solver = step.module(NextSig, **(step.kwargs or {}))
        result = step_solver(**input_vals)
        for o in step.outputs:
            input_vals[o.name] = result[o.name]
        input_sigs.extend(step.outputs)
        solvers[step_name] = step_solver
        results[step_name] = result
    return solvers, results

In [34]:
pdf_text = pdf_utils.extract_text_from_pdf("papers/94cifar.pdf")

In [79]:
solvers, results = solve_sigchain(paper_chain, {"paper_content": pdf_text})

Running step title_abstract_plus
  Inputs: paper_content
  Outputs: title, abstract_plus
Running step core_ideas
  Inputs: paper_content, title, abstract_plus
  Outputs: core_ideas
Running step metrics
  Inputs: paper_content, title, abstract_plus, core_ideas
  Outputs: metrics
Running step baseline_methods
  Inputs: paper_content, title, abstract_plus, core_ideas, metrics
  Outputs: baseline_methods
Running step experimental_methods
  Inputs: paper_content, title, abstract_plus, core_ideas, metrics, baseline_methods
  Outputs: experimental_methods
Running step method_metric_results
  Inputs: paper_content, title, abstract_plus, core_ideas, metrics, baseline_methods, experimental_methods
  Outputs: method_metric_results
Running step hw_agnostic_metrics
  Inputs: paper_content, title, abstract_plus, core_ideas, metrics, baseline_methods, experimental_methods, method_metric_results
  Outputs: hw_agnostic_metrics
Running step tasks
  Inputs: paper_content, title, abstract_plus, core_ideas

In [80]:
records = {}
records["v4"] = (solvers, results)

In [66]:
print(results['task_eval_baseline_implementation']['task_eval_baseline_implementation'])

Since no specific task was requested for a baseline implementation, I'll provide a general response:

For most of the tasks outlined in this paper, a baseline implementation would not be appropriate or necessary. The tasks described are primarily about implementing novel methods or components of the airbench system, rather than improving upon existing baselines.

However, if we were to consider a baseline for the overall training method (task 6: train_airbench94), a simple baseline could be a standard CNN training on CIFAR-10 without the advanced techniques introduced in the paper. Here's a basic outline of such a baseline:

```python
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Define a simple CNN
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        

In [51]:
print(results['setup']['setup'])

1. Python environment with PyTorch (version 2.1.2 or newer) and CUDA support
2. CUDA-capable GPU
3. Python libraries: torch, torchvision
4. CIFAR-10 dataset (will be downloaded automatically by torchvision)
5. Configuration dictionary for hyperparameters
6. Modules for:
   - Network architecture (VGG-like with modifications)
   - Data loading with custom augmentations (including alternating flip)
   - Training loop with optimization techniques (Nesterov SGD, Lookahead, etc.)
   - Evaluation function with test-time augmentation options
7. Logging functionality for tracking training progress and results

With this setup, we should be able to implement and run the airbench training methods described in the paper. The CIFAR-10 dataset will be automatically downloaded when first used, and the custom network and training procedures can be implemented using the provided PyTorch environment.


In [81]:
for k in results.keys():
    for kk in results[k].keys():
        if kk == "rationale":
            continue
        print(f"{kk}:\n{results[k][kk]}\n\n")
    print ("\n\n====================\n\n")

title:
94% on CIFAR-10 in 3.29 Seconds on a Single GPU


abstract_plus:
This paper introduces ultra-fast training methods for the CIFAR-10 dataset, achieving 94% accuracy in just 3.29 seconds on a single NVIDIA A100 GPU. This represents a 1.9x speedup over the previous state-of-the-art. The authors also develop methods targeting 95% accuracy in 10.4 seconds and 96% accuracy in 46.3 seconds. A key innovation is the introduction of "alternating flip" data augmentation, which improves upon standard random flipping in most scenarios. The training methods are released as open-source code.

The paper provides a detailed analysis of how different components contribute to the overall speedup, including network architecture choices, optimization techniques, and the novel data augmentation. Experiments demonstrate that the developed methods generalize well to other datasets like CIFAR-100 and SVHN without additional tuning. The authors also investigate statistical properties of their training ap

In [ ]:
class PaperAnalysis(dspy.Signature):
    """Analyze a research paper and provide core information."""

    paper_content = dspy.InputField(desc="The full text content of the research paper")

    title = dspy.OutputField(desc="The title of the paper")
    summary = dspy.OutputField(desc="A concise summary of the paper's main contributions")
    core_ideas = dspy.OutputField(desc="The core idea(s) of the paper")
    methods = dspy.OutputField(desc="Key methods or strategies proposed in the paper")
    metrics = dspy.OutputField(desc="Primary metrics or evaluation criteria used in the paper")
    requirements = dspy.OutputField(desc="Key requirements for implementing and evaluating the paper's methods")

In [14]:
def configure_lm():
    """Configure and return a language model."""
    lm = dspy.Claude("claude-3-5-sonnet-20240620", api_key="***REMOVED***")
    dspy.configure(lm=lm)
    return lm

def analyze_paper(pdf_path, model_name="gpt-3.5-turbo"):
    """Analyze a research paper given its PDF path."""

    # Configure LM
    lm = configure_lm()

    # Extract text from PDF
    paper_content = pdf_utils.extract_text_from_pdf(pdf_path)
 
    # Initialize and compile the analyzer
    analyzer = dspy.ChainOfThought(paper_analysis.PaperAnalysis)
    # Analyze the paper
    result = analyzer(paper_content=paper_content)

    return result

In [15]:
result=analyze_paper("papers/94cifar.pdf")

In [22]:
result.keys()

['rationale', 'summary', 'core_ideas', 'methods', 'metrics', 'requirements']

In [26]:
result.summary

str

In [24]:
result.methods

'1. Modified network architecture based on prior work, with some adjustments to layer sizes and initialization.\n2. Frozen patch-whitening initialization for the first convolutional layer.\n3. Identity initialization for subsequent convolutional layers.\n4. Optimization tricks including increased learning rate for BatchNorm biases and Lookahead optimization.\n5. Multi-crop evaluation using six augmented views of each test image.\n6. Alternating flip augmentation technique.\n7. Compilation using torch.compile for improved GPU utilization.'

In [25]:
result.metrics

'1. Training time to reach target accuracy (94%, 95%, or 96%)\n2. Number of FLOPs (Floating Point Operations) required\n3. Test set accuracy\n4. Class-aggregated calibration error (CACE)'